## Poligony wojskowe w Polsce

In [1]:
#!pip install beautifulsoup4
#!pip install lxml==4.6.4
#!pip install html5lib
#!pip install geopy

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
## Webscrapping page with BS
url = "https://pl.wikipedia.org/wiki/Poligony_wojskowe_w_Polsce"
data = requests.get(url).text
soup = BeautifulSoup(data, "html.parser")
tables = soup.find_all('table')

In [4]:
#Creating my intial DataFrame
poligony_data = pd.DataFrame(columns = ["Poligon", "Lokalizacja", "Status"])
for row in tables[1].find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        poligon = col[0].text.strip()
        lok = col[1].text.strip()
        sts = col[2].text.strip()
        poligony_data = poligony_data.append({"Poligon": poligon, "Lokalizacja": lok, "Status": sts}, ignore_index = True) 


C:\Users\Olga\AppData\Local\Temp\ipykernel_19696\151299551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  poligony_data = poligony_data.append({"Poligon": poligon, "Lokalizacja": lok, "Status": sts}, ignore_index = True)
C:\Users\Olga\AppData\Local\Temp\ipykernel_19696\151299551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  poligony_data = poligony_data.append({"Poligon": poligon, "Lokalizacja": lok, "Status": sts}, ignore_index = True)
C:\Users\Olga\AppData\Local\Temp\ipykernel_19696\151299551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  poligony_data = poligony_data.append({"Poligon": poligon, "Lokalizacja": lok, "Status": sts}, ignore_index = True)
C:\Users\Olga\AppData\Local\Temp\ipykernel_19696\151299

In [5]:
#creating new DF with poligons being active
czynne_poligony = (poligony_data.loc[(poligony_data['Status'] == 'czynny') | (poligony_data['Status'] == 'czynny czasowo') ])


In [6]:
#prevent SettingWithCopyWarning message from appearing
pd.options.mode.chained_assignment = None
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="GetLoc")

#Adding two new columns. 1 finding address with geocode. 2 isolating coordinates

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
czynne_poligony.loc[:,"Adres"] = czynne_poligony.loc[:,"Poligon"].apply(geocode)

czynne_poligony.loc[:,"Punkt"] = czynne_poligony.loc[:,"Adres"].apply(lambda loc: list(loc.point) if loc else None)


In [7]:
#just as an exercise it could stay NaN as well
values = {"Adres": "brak" , "Punkt" : "brak"}
czynne_poligony = czynne_poligony.fillna(value = values)


In [8]:
#removing 3rd coordinate from Punkt,it is not useful
czynne_poligony["Punkt"] = czynne_poligony["Punkt"].map((lambda coord : (coord[:-1]) if coord != "brak" else "brak"))

In [9]:
#just to make it look nicer, so index does not look confusing
idx = pd.Index(range(16))
czynne_poligony = czynne_poligony.set_index(idx)


In [10]:
# creating two last columns baded on Punkt column. Unfortunetly Lipa ends up in Philippines
czynne_poligony['Szerokosc'] = czynne_poligony.apply(lambda row: row['Punkt'][0], axis = 1)
czynne_poligony['Dlugosc'] = czynne_poligony.apply(lambda row: row['Punkt'][1], axis = 1)
czynne_poligony

,Poligon,Lokalizacja,Status,Adres,Punkt,Szerokosc,Dlugosc
0,Biedrusko,"na północ od miasta Poznań, między Biedruskiem...",czynny,"(Biedrusko, gmina Suchy Las, powiat poznański,...","[52.5432315, 16.9458753]",52.543231,16.945875
1,Czarne,na północ i południowy zachód od miasta Czarne,czynny,"(Czarne, gmina Czarne, powiat człuchowski, woj...","[53.68353, 16.94221]",53.68353,16.94221
2,Drawsko Pomorskie,na południe od miasta Drawsko Pomorskie,czynny,"(Drawsko Pomorskie, gmina Drawsko Pomorskie, p...","[53.5310786, 15.8134179]",53.531079,15.813418
3,Jagodne,"w Lasach Łukowskich, na północny zachód od mia...",czynny,"(Jagodne, gmina Pisz, powiat piski, województw...","[53.61394, 21.8325]",53.61394,21.8325
4,Lipa,na północny zachód od wsi Lipa,czynny czasowo,"(Lipa, Batangas, Calabarzon, 4217, Philippines...","[13.9414686, 121.1642877]",13.941469,121.164288
5,Nadarzyce,na północny zachód od wsi Nadarzyce,czynny,"(Nadarzyce, gmina Września, powiat wrzesiński,...","[52.2971846, 17.6122521]",52.297185,17.612252
6,Nowa Dęba,na wschód od miasta Nowa Dęba,czynny,"(Nowa Dęba, gmina Nowa Dęba, powiat tarnobrzes...","[50.4132684, 21.749056]",50.413268,21.749056
7,Orzysz,na południowy wschód od miasta Orzysz,czynny,"(Orzysz, gmina Orzysz, powiat piski, województ...","[53.809833, 21.9473601]",53.809833,21.94736
8,Radomyśl nad Sanem,"na Sanie, niedaleko ujścia do Wisły",czynny czasowo,"(Radomyśl nad Sanem, gmina Radomyśl nad Sanem,...","[50.6805533, 21.947306]",50.680553,21.947306
9,Strzepcz,"na południowy wschód od miasta Lębork, między ...",czynny,"(Strzepcz, gmina Linia, powiat wejherowski, wo...","[54.454199, 18.0262813]",54.454199,18.026281


###### 

In [11]:
#!pip install folium

In [12]:
import folium

In [13]:
m = folium.Map(location=[52.27696486537947, 20.960099173420097], zoom_start=6)

tooltip = "Kliknij"

for i in range(0, len(czynne_poligony)):
    if (czynne_poligony.iloc[i]['Szerokosc'] != 'b') and (czynne_poligony.iloc[i]['Dlugosc'] != 'r'):
        folium.Marker(location = [czynne_poligony.iloc[i]['Szerokosc'], czynne_poligony.iloc[i]['Dlugosc']],
          popup=czynne_poligony.iloc[i]['Poligon'],
       ).add_to(m)


m